BENGOUNIA Mahmoud

FERRAH Hichem

In [ ]:
!pwd
!ls ../
!pwd ../

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
'''
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
root_path = '/kaggle/input/CORD-19-research-challenge/'
metadata_path = f'{root_path}/metadata.csv'
meta_df = pd.read_csv(metadata_path, dtype={
    'pubmed_id': str,
    'Microsoft Academic Paper ID': str, 
    'doi': str
})
meta_df.head(2)
meta_df.info()

In [ ]:
import glob
import json
pdf_json = glob.glob(f'{root_path}document_parses/pdf_json/*.json', recursive=True)
pmc_json = glob.glob(f'{root_path}document_parses/pmc_json/*.json', recursive=True)
all_json = pdf_json
all_json.append(pmc_json)
len(all_json)

In [ ]:
#we transform each file into a line in our dataset
def FileToLine(path):
    line=[]
    json_content= None
    with open(path) as json_file:
        json_content=json.load(json_file)
        
    paper_id=json_content['paper_id'] #... ID
    
    meta_data = meta_df.loc[meta_df['sha'] == paper_id]
    if len(meta_data)==0: return None #... Return Nothing / skip paper
    
    #Added the index of each column in a comment so it can be easly seen
    line.append(paper_id) #...0        
    abstract=[]
        
    for paragraph in json_content['abstract']: #... Abstract
        abstract.append(paragraph['text'])
    abstract=';'.join(abstract)
    line.append(abstract) #...1
    body=[]
        
    for paragraph in json_content['body_text']:#... Body Content by Paragraph
        body.append(paragraph['text'])
    body=';'.join(body) # Hopefully ';' doesn't mess up our file like '\n' did
    line.append(body) #...2
        
    line.append(meta_data['authors'].values[0])  #...3
    line.append(meta_data['title'].values[0]) #...4
    line.append(meta_data['journal'].values[0]) #..5
    line.append(meta_data['doi'].values[0]) #..6
    
    return line
#print(FileToLine(all_json[0]))
a=FileToLine(all_json[0])
for i in a:
    print(type(i))


In [ ]:
import time

In [ ]:
'''
data= {'paper_id': [], 'doi':[], 'abstract': [], 'body_text': [], 'authors': [], 'title': [], 'journal': []} #data_set format
#some values to track our progression
total_entries=len(all_json)
number_entries=total_entries//10
#number_entries=20 #test-functionaliy
part=1
start_time=time.time()
list=['paper_id','abstract','body_text','authors','title','journal','doi']
#we could have done everything in one bloc, but we might need every part indepandently later for improvements
for idx, entry in enumerate(all_json):
    elapsed_time=time.time()-start_time
    minutes=round(elapsed_time // 60)
    seconds=round(elapsed_time % 60)
    print(f'......Processing index: {idx} of {total_entries} -°o0o°- {number_entries} left for part N°{part} ... {minutes} mins {seconds} secs passed......',end='\r')
    try:
        line =FileToLine(entry)
    except: # Unknown File
        continue
    if line == None: continue
    
    #line[i]-->0.paper_id 1.abstract 2.body 3.authors 4.title 5.journal 6.doi
    for i in range(len(line)):
        if type(line[i]) == type("this is a string"):
            data[list[i]].append(line[i].replace(","," ")) # we delete the commas so they can be our seperator in the data frame we will save
        else:
            data[list[i]].append(line[i])
    number_entries-=1
    if number_entries <= 0: #saving our data set while making sure that it's divided into parts so we can 
        print(f'\n saving part{part}...')
        df_covid = pd.DataFrame(data, columns=['paper_id', 'doi', 'abstract', 'body_text', 'authors', 'title', 'journal'])
        df_covid.to_csv (f'all_CORD_data_part{part}.csv', index = False, header=True)
        print(f"part{part} saved!")
        data= {'paper_id': [], 'doi':[], 'abstract': [], 'body_text': [], 'authors': [], 'title': [], 'journal': []} #data_set format
        number_entries=total_entries//10
        #number_entries=20 #test
        part+=1
    ###SINCE IT TAKES A LOT OF TIME TO PROCESS ALL ENTRIES WE WILL JUST GONNA PICK THE FIRST PART TO SPEED WORK, DELETE/COMMENT THE NEXT PART FOR THE REAL EXECUTION:
    #if part >1: break # <- Delete this and "part > 2" below
        
if number_entries >0: #remaining entries in the last part
    print(f'saving part{part}...')
    df_covid = pd.DataFrame(data, columns=['paper_id', 'doi', 'abstract', 'body_text', 'authors', 'title', 'journal', 'abstract_summary'])
    df_covid.to_csv (f'all_CORD_data_part{part}.csv', index = False, header=True)
    print(f"part{part} saved!")
print(f"Success! all data saved into {part} parts")
        
'''

In [ ]:
data= {'paper_id': [], 'doi':[], 'abstract': [], 'body_text': [], 'authors': [], 'title': [], 'journal': []} #data_set format
#some values to track our progression
total_entries=len(all_json)
#number_entries=total_entries//10
#part=1
start_time=time.time()
list=['paper_id','abstract','body_text','authors','title','journal','doi']
#we could have done everything in one bloc, but we might need every part indepandently later for improvements
for idx, entry in enumerate(all_json):
    elapsed_time=time.time()-start_time
    minutes=round(elapsed_time // 60)
    seconds=round(elapsed_time % 60)
    print(f'......Processing index: {idx} of {total_entries} -°o0o°-... {minutes} mins {seconds} secs passed......',end='\r')
    try:
        line =FileToLine(entry)
    except: # Unknown File
        continue
    if line == None: continue
    
    #line[i]-->0.paper_id 1.abstract 2.body 3.authors 4.title 5.journal 6.doi
    for i in range(len(line)):
        if type(line[i]) == type("this is a string"):
            data[list[i]].append(line[i].replace(","," ")) # we delete the commas so they can be our seperator in the data frame we will save
        else:
            data[list[i]].append(line[i])
    #number_entries-=1
print(f'\n saving File...')
df_covid = pd.DataFrame(data, columns=['paper_id', 'doi', 'abstract', 'body_text', 'authors', 'title', 'journal'])
df_covid.to_csv (f'all_CORD_data_part.csv', index = False, header=True)
print(f"Saved")
#data= {'paper_id': [], 'doi':[], 'abstract': [], 'body_text': [], 'authors': [], 'title': [], 'journal': []} #data_set format
#number_entries=total_entries//10

In [ ]:
df_covid.head()